In [ ]:
# 1

# Dawson Field

In [5]:
# 2

import os
import pandas as pd


path = "/Users/dawson/Dropbox/5330corgis"

files = os.listdir(path)


print("Files in the 5330corgis directory:")
for file in files:
    print(file)


Files in the 5330corgis directory:
pdx623.csv
portlandeventjun22.csv
newreg-062822.csv
Icon
newcorgis-20220619.csv
newcorgis-20220618.csv
signupsjune272022.csv
tacoma-062322.csv


In [9]:
# 3

df = pd.read_csv('/Users/dawson/Dropbox/5330corgis/pdx623.csv')
print(df)

                            Name Breed Gender  Weight  Age
0  Wendell of the Gnashing Teeth   CWC     NM      32  3.3
1                Trotsky the Dog   Pem     SF      23  5.5
2                CanCan of Beans   Pem     SF      25  4.2
3             Harvey the Younger   PWC      M      22  1.1
4                 Jenni the Shiv   PWC     SF      28  3.0
5           Ogbert the Semi-Wise   Pem     NM      25  4.0
6                Jeff OCorgosaur   PWC      M      27  2.0
7                  Poofy Mudfoot   Pem     SF      23  2.1


In [61]:
# 4

import psycopg2

conn = psycopg2.connect(
    database='corgirace',
    user='dfield7',
    password='goaggies!',
    host='dawson-database.c96so8s4mt3y.us-east-2.rds.amazonaws.com',
    port='5432'
)
cursor = conn.cursor()

#crearting lists
csvlist = []
need_to_ingest = []

# for loop for csvs and ones that need to be processed
for file in files:
    if file[-3:]!= 'csv':
        print('has been skipped',file)
        print('\n')
        continue
    else:
        print('is csv',file)
        
    csvlist.append(file)

    cursor.execute("""
        SELECT COUNT(*)
        FROM ingest
        WHERE filename = %s
    """, (file,))

    y = cursor.fetchone()
    if y[0] > 0:
        print('already processed',file)
    else:
        print('not in yet',file)
        need_to_ingest.append(file)
        print('\n')

print(csvlist)

#fies that need to be processed
for x in need_to_ingest:
    print("this file needs to be processd:",x)

for file in need_to_ingest:
    filename = path + '/' + file
    print(filename)
    df = pd.read_csv(filename)
    print(df)

    # Apply changes to the dataframe
    df['Breed'].replace(to_replace=['PWC', 'CWC'], value=['Pem', 'Cardi'], inplace=True)
    df['Gender'].replace(to_replace='Fem', value='F', inplace=True)
    df['Weight'] = df['Weight'].round() 
    df['Age'] = df['Age'].round(1)

    # Insert each row into the corgi table using f-strings
    for x in df.index:
        cursor.execute(f'''
            INSERT INTO corgi (corgname, Breed, Gender, Weight, Age, fromfile)
            VALUES ('{df["Name"].loc[x]}', '{df["Breed"].loc[x]}', '{df["Gender"].loc[x]}',
                    {df["Weight"].loc[x]}, {df["Age"].loc[x]}, '{file}')
        ''')
        conn.commit() 
        print('Record created for %s' % df['Name'].loc[x])

        # Insert the file into the ingest table after processing
    cursor.execute(f'''
        INSERT INTO ingest (filename, whendone)
        VALUES ('{file}', current_timestamp)
    ''')
    conn.commit()
    
    print(f'Ingestion complete for {file}')

is csv pdx623.csv
not in yet pdx623.csv


is csv portlandeventjun22.csv
not in yet portlandeventjun22.csv


is csv newreg-062822.csv
not in yet newreg-062822.csv


has been skipped Icon


is csv newcorgis-20220619.csv
not in yet newcorgis-20220619.csv


is csv newcorgis-20220618.csv
already processed newcorgis-20220618.csv
is csv signupsjune272022.csv
not in yet signupsjune272022.csv


is csv tacoma-062322.csv
not in yet tacoma-062322.csv


['pdx623.csv', 'portlandeventjun22.csv', 'newreg-062822.csv', 'newcorgis-20220619.csv', 'newcorgis-20220618.csv', 'signupsjune272022.csv', 'tacoma-062322.csv']
this file needs to be processd: pdx623.csv
this file needs to be processd: portlandeventjun22.csv
this file needs to be processd: newreg-062822.csv
this file needs to be processd: newcorgis-20220619.csv
this file needs to be processd: signupsjune272022.csv
this file needs to be processd: tacoma-062322.csv
/Users/dawson/Dropbox/5330corgis/pdx623.csv
                            Name Breed Gende

/var/folders/4s/t6sswdb1013_0mcrz0gvj5k40000gn/T/ipykernel_3174/35192460.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Breed'].replace(to_replace=['PWC', 'CWC'], value=['Pem', 'Cardi'], inplace=True)
/var/folders/4s/t6sswdb1013_0mcrz0gvj5k40000gn/T/ipykernel_3174/35192460.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on 

Record created for Trotsky the Dog
Record created for CanCan of Beans
Record created for Harvey the Younger
Record created for Jenni the Shiv
Record created for Ogbert the Semi-Wise
Record created for Jeff OCorgosaur
Record created for Poofy Mudfoot
Ingestion complete for pdx623.csv
                            Name  Breed Gender  Weight  Age
0  Wendell of the Gnashing Teeth  Cardi     NM      32  3.3
1                Trotsky the Dog    Pem     SF      23  5.5
2                CanCan of Beans    Pem     SF      25  4.2
3             Harvey the Younger    Pem      M      22  1.1
4                 Jenni the Shiv    Pem     SF      28  3.0
5           Ogbert the Semi-Wise    Pem     NM      25  4.0
6                Jeff OCorgosaur    Pem      M      27  2.0
7                  Poofy Mudfoot    Pem     SF      23  2.1
/Users/dawson/Dropbox/5330corgis/portlandeventjun22.csv
                     Name  Breed Gender  Weight  Age
0      Amber Waves of Fur    PWC     SF      28  4.0
1        Kodla

/var/folders/4s/t6sswdb1013_0mcrz0gvj5k40000gn/T/ipykernel_3174/35192460.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Breed'].replace(to_replace=['PWC', 'CWC'], value=['Pem', 'Cardi'], inplace=True)
/var/folders/4s/t6sswdb1013_0mcrz0gvj5k40000gn/T/ipykernel_3174/35192460.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on 

Record created for Amber Waves of Fur
Record created for Kodlak Whitemane
Record created for Floaty Bee
Record created for Bag o Donuts
Record created for Hot Ham and Cheese
Record created for Cruzer the Cruzedog
Record created for Sally the Destroyer
Record created for Elma Redfur
Record created for Kool Thing
Record created for Skitters McDogdog
Record created for Ophelio
Record created for Mr. Stealyoursandwich
Ingestion complete for portlandeventjun22.csv
                     Name  Breed Gender  Weight  Age
0      Amber Waves of Fur    Pem     SF      28  4.0
1        Kodlak Whitemane    Pem      M      27  2.2
2              Floaty Bee    Pem      F      23  2.5
3            Bag o Donuts    Pem     SF      28  3.8
4      Hot Ham and Cheese  Cardi     NM      29  4.0
5     Cruzer the Cruzedog    Pem      M      23  2.5
6     Sally the Destroyer    Pem      F      29  5.2
7             Elma Redfur    Pem     SF      26  2.0
8              Kool Thing  Cardi      F      29  3.5
9     

...

Record created for Splüüter
Record created for Vampira
Record created for Heracles
Record created for Spatula
Record created for Maisey the Faerie Empress
Ingestion complete for newcorgis-20220619.csv
                        Name  Breed Gender  Weight  Age
0                   Splüüter    Pem     NM      26  2.2
1                    Vampira  Cardi      F      28  3.0
2                   Heracles    Pem      M      29  4.8
3                    Spatula    Pem      F      26  4.0
4  Maisey the Faerie Empress    Pem     SF      20  1.5
/Users/dawson/Dropbox/5330corgis/signupsjune272022.csv
               Name Breed Gender  Weight  Age
0        Cashew Nut   PWC      F      25  4.5
1  Grover Cleveland   Pem      M      26  3.0
2           Treacle   PWC      F      23  2.5
3      Kraft Dinner   PWC     SF      23  2.2
4  Ryebread Megadog   PWC     NM      31  5.0
5    Courtesy Check   Pem      F      22  3.0


/var/folders/4s/t6sswdb1013_0mcrz0gvj5k40000gn/T/ipykernel_3174/35192460.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Breed'].replace(to_replace=['PWC', 'CWC'], value=['Pem', 'Cardi'], inplace=True)
/var/folders/4s/t6sswdb1013_0mcrz0gvj5k40000gn/T/ipykernel_3174/35192460.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on 

Record created for Cashew Nut
Record created for Grover Cleveland
Record created for Treacle
Record created for Kraft Dinner
Record created for Ryebread Megadog
Record created for Courtesy Check
Ingestion complete for signupsjune272022.csv
               Name Breed Gender  Weight  Age
0        Cashew Nut   Pem      F      25  4.5
1  Grover Cleveland   Pem      M      26  3.0
2           Treacle   Pem      F      23  2.5
3      Kraft Dinner   Pem     SF      23  2.2
4  Ryebread Megadog   Pem     NM      31  5.0
5    Courtesy Check   Pem      F      22  3.0
/Users/dawson/Dropbox/5330corgis/tacoma-062322.csv
               Name  Breed Gender  Weight  Age
0            Dogpup    PWC      M      25  3.5
1  Oscar the Gaucho    CWC     NM      32  2.1
2         Cleopupra    PWC      F      24  2.0
3        Briar Rose    PWC     SF      26  5.4
4  Flannery ODogger    PWC     SF      20  3.0
5          Facepalm    Pem      M      27  2.5
6           Mugwort  Cardi     NM      26  4.0
Record crea

/var/folders/4s/t6sswdb1013_0mcrz0gvj5k40000gn/T/ipykernel_3174/35192460.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Breed'].replace(to_replace=['PWC', 'CWC'], value=['Pem', 'Cardi'], inplace=True)
/var/folders/4s/t6sswdb1013_0mcrz0gvj5k40000gn/T/ipykernel_3174/35192460.py:57: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on 

Record created for Oscar the Gaucho
Record created for Cleopupra
Record created for Briar Rose
Record created for Flannery ODogger
Record created for Facepalm
Record created for Mugwort
Ingestion complete for tacoma-062322.csv
               Name  Breed Gender  Weight  Age
0            Dogpup    Pem      M      25  3.5
1  Oscar the Gaucho  Cardi     NM      32  2.1
2         Cleopupra    Pem      F      24  2.0
3        Briar Rose    Pem     SF      26  5.4
4  Flannery ODogger    Pem     SF      20  3.0
5          Facepalm    Pem      M      27  2.5
6           Mugwort  Cardi     NM      26  4.0


In [63]:
# 5

query = ('''
    SELECT * 
    FROM ingest;
    ''')
         
ingest_df = pd.read_sql(query, conn)
print(ingest_df)

/var/folders/4s/t6sswdb1013_0mcrz0gvj5k40000gn/T/ipykernel_3174/2165624274.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ingest_df = pd.read_sql(query, conn)


                 filename                   whendone
0  newcorgis-20220618.csv 2024-10-11 19:50:16.969221
1              pdx623.csv 2024-10-19 00:02:19.607316
2  portlandeventjun22.csv 2024-10-19 00:02:21.915258
3       newreg-062822.csv 2024-10-19 00:02:22.884741
4  newcorgis-20220619.csv 2024-10-19 00:02:23.915843
5   signupsjune272022.csv 2024-10-19 00:02:25.259272
6       tacoma-062322.csv 2024-10-19 00:02:27.419275


In [87]:
# 6

from datetime import datetime

query = """
    SELECT c.corgiid, c.corgname, c.fromfile, 
           TO_CHAR(i.whendone, 'YYYY-MM-DD') AS whendone_date 
    FROM corgi c
    JOIN ingest i ON c.fromfile = i.filename
    WHERE DATE_PART('month', i.whendone) = DATE_PART('month', CURRENT_DATE)
    ORDER BY c.corgname;
"""

df_corgi = pd.read_sql(query, conn, index_col='corgiid')
print(df_corgi)

                              corgname                fromfile whendone_date
corgiid                                                                     
43                  Amber Waves of Fur  portlandeventjun22.csv    2024-10-19
46                        Bag o Donuts  portlandeventjun22.csv    2024-10-19
59                 Betty Doggy Sprinty       newreg-062822.csv    2024-10-19
74                          Briar Rose       tacoma-062322.csv    2024-10-19
37                     CanCan of Beans              pdx623.csv    2024-10-19
65                          Cashew Nut   signupsjune272022.csv    2024-10-19
73                           Cleopupra       tacoma-062322.csv    2024-10-19
70                      Courtesy Check   signupsjune272022.csv    2024-10-19
48                 Cruzer the Cruzedog  portlandeventjun22.csv    2024-10-19
71                              Dogpup       tacoma-062322.csv    2024-10-19
50                         Elma Redfur  portlandeventjun22.csv    2024-10-19

/var/folders/4s/t6sswdb1013_0mcrz0gvj5k40000gn/T/ipykernel_3174/3301451973.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_corgi = pd.read_sql(query, conn, index_col='corgiid')
